# Customer Cohort & Lifetime Value Analysis

**Purpose:** Deep dive into customer cohorts, retention patterns, and lifetime value calculations.

**Key Questions:**
- Which cohorts have the highest retention?
- What is the estimated customer lifetime value by segment?
- When do customers typically churn?
- Which acquisition months produced the best customers?

---

In [ ]:
# Setup
import os
import sys
from pathlib import Path

# Add project root to path for importing local modules
project_root = Path().absolute().parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from google.cloud import bigquery
from google.oauth2 import service_account

sns.set_style("whitegrid")
plt.rcParams["figure.figsize"] = (14, 8)

# Try to import config from project utils
try:
    from src.utils.config import get_config

    config = get_config()
    project_id = config.gcp_project_id
    credentials_path = config.google_application_credentials
except ImportError:
    # Fallback to environment variables
    credentials_path = os.getenv("GOOGLE_APPLICATION_CREDENTIALS")
    project_id = os.getenv("GCP_PROJECT_ID", "project-samba-insight")

# Get dataset name from environment or use default
dataset = os.getenv("BQ_DATASET_WAREHOUSE", "dev_warehouse_warehouse")

# Initialize BigQuery
if not credentials_path or not os.path.exists(credentials_path):
    print("⚠️  Warning: Using application default credentials")
    client = bigquery.Client(project=project_id)
else:
    credentials = service_account.Credentials.from_service_account_file(
        credentials_path,
        scopes=["https://www.googleapis.com/auth/bigquery"],
    )
    client = bigquery.Client(project=project_id, credentials=credentials)

print(f"✅ Connected to project: {project_id}")
print(f"📊 Using dataset: {dataset}")

## 1. Cohort Overview

Load cohort data and understand cohort composition.

In [ ]:
query_cohorts = f"""
SELECT
  cohort_month,
  cohort_month_name,
  customer_state,
  cohort_size,
  loyal_customers,
  repeat_customers,
  one_time_customers,
  customer_ltv,
  avg_orders_per_customer,
  total_revenue,
  avg_review_score,
  loyal_pct,
  repeat_pct,
  one_time_pct
FROM `{project_id}.{dataset}.mart_customer_cohorts`
WHERE cohort_month >= '2017-01-01'
ORDER BY cohort_month, customer_state
"""

df_cohorts = client.query(query_cohorts).to_dataframe()

print(f"📊 Loaded {len(df_cohorts)} cohort records")
print(f"📅 Date Range: {df_cohorts['cohort_month'].min()} to {df_cohorts['cohort_month'].max()}")
print(f"👥 Total Customers Analyzed: {df_cohorts['cohort_size'].sum():,}")

## 2. Lifetime Value by Cohort

Compare LTV across acquisition cohorts.

In [ ]:
# Aggregate by cohort month (across all states)
cohort_summary = (
    df_cohorts.groupby("cohort_month_name")
    .agg(
        {
            "cohort_size": "sum",
            "customer_ltv": "mean",
            "avg_orders_per_customer": "mean",
            "total_revenue": "sum",
            "avg_review_score": "mean",
        }
    )
    .reset_index()
)

# Plot LTV by cohort
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10))

# LTV trend
ax1.bar(
    cohort_summary["cohort_month_name"], cohort_summary["customer_ltv"], color="#2ca02c", alpha=0.7
)
ax1.axhline(
    y=cohort_summary["customer_ltv"].mean(),
    color="red",
    linestyle="--",
    label=f"Average: R${cohort_summary['customer_ltv'].mean():.2f}",
)
ax1.set_title("Customer Lifetime Value by Acquisition Cohort", fontsize=16, fontweight="bold")
ax1.set_ylabel("LTV (R$)", fontsize=12)
ax1.legend()
ax1.grid(True, alpha=0.3, axis="y")
ax1.tick_params(axis="x", rotation=45)

# Cohort size trend
ax2.plot(
    cohort_summary["cohort_month_name"],
    cohort_summary["cohort_size"],
    marker="o",
    linewidth=2,
    color="#1f77b4",
)
ax2.set_title("Cohort Size Over Time", fontsize=16, fontweight="bold")
ax2.set_ylabel("Customers", fontsize=12)
ax2.set_xlabel("Cohort Month", fontsize=12)
ax2.grid(True, alpha=0.3)
ax2.tick_params(axis="x", rotation=45)

plt.tight_layout()
plt.show()

print("\n🏆 Top 5 Cohorts by LTV:")
top_ltv = cohort_summary.nlargest(5, "customer_ltv")[
    ["cohort_month_name", "customer_ltv", "cohort_size"]
]
print(top_ltv.to_string(index=False))

## 3. Customer Segment Distribution by Cohort

Analyze how segment distribution varies across cohorts.

In [ ]:
# Calculate segment percentages
cohort_summary["loyal_pct"] = df_cohorts.groupby("cohort_month_name")["loyal_pct"].mean().values
cohort_summary["repeat_pct"] = df_cohorts.groupby("cohort_month_name")["repeat_pct"].mean().values
cohort_summary["one_time_pct"] = (
    df_cohorts.groupby("cohort_month_name")["one_time_pct"].mean().values
)

# Stacked bar chart
fig, ax = plt.subplots(figsize=(14, 7))

x = np.arange(len(cohort_summary))
width = 0.8

p1 = ax.bar(x, cohort_summary["loyal_pct"], width, label="Loyal (3+ orders)", color="#2ca02c")
p2 = ax.bar(
    x,
    cohort_summary["repeat_pct"],
    width,
    bottom=cohort_summary["loyal_pct"],
    label="Repeat (2 orders)",
    color="#ff7f0e",
)
p3 = ax.bar(
    x,
    cohort_summary["one_time_pct"],
    width,
    bottom=cohort_summary["loyal_pct"] + cohort_summary["repeat_pct"],
    label="One-time (1 order)",
    color="#d62728",
)

ax.set_ylabel("Percentage", fontsize=12)
ax.set_title("Customer Segment Distribution by Cohort", fontsize=16, fontweight="bold")
ax.set_xticks(x)
ax.set_xticklabels(cohort_summary["cohort_month_name"], rotation=45, ha="right")
ax.legend()
ax.grid(True, alpha=0.3, axis="y")

plt.tight_layout()
plt.show()

print("\n💡 Segment Insights:")
print(f"  • Average Loyal: {cohort_summary['loyal_pct'].mean():.1f}%")
print(f"  • Average Repeat: {cohort_summary['repeat_pct'].mean():.1f}%")
print(f"  • Average One-time: {cohort_summary['one_time_pct'].mean():.1f}%")

## 4. Retention Curves by Cohort

Detailed retention analysis showing how different cohorts retain customers over time.

In [ ]:
query_retention = f"""
SELECT
  cohort_month_name,
  months_since_cohort,
  retention_rate_pct,
  active_customers,
  cohort_size
FROM `{project_id}.{dataset}.mart_customer_retention`
WHERE cohort_month >= '2017-01-01'
  AND cohort_month <= '2017-06-01'  -- First 6 cohorts
  AND months_since_cohort <= 12
ORDER BY cohort_month, months_since_cohort
"""

df_retention = client.query(query_retention).to_dataframe()

# Plot retention curves
fig, ax = plt.subplots(figsize=(14, 8))

for cohort in df_retention["cohort_month_name"].unique():
    cohort_data = df_retention[df_retention["cohort_month_name"] == cohort]
    ax.plot(
        cohort_data["months_since_cohort"],
        cohort_data["retention_rate_pct"],
        marker="o",
        linewidth=2,
        label=cohort,
    )

ax.set_xlabel("Months Since First Order", fontsize=12)
ax.set_ylabel("Retention Rate (%)", fontsize=12)
ax.set_title("Customer Retention Curves by Cohort", fontsize=16, fontweight="bold")
ax.legend(title="Cohort", bbox_to_anchor=(1.05, 1), loc="upper left")
ax.grid(True, alpha=0.3)
ax.set_ylim(0, 100)

plt.tight_layout()
plt.show()

# Calculate average retention by month
avg_retention = df_retention.groupby("months_since_cohort")["retention_rate_pct"].mean()
print("\n📊 Average Retention Rates:")
for month in [1, 3, 6, 12]:
    if month in avg_retention.index:
        print(f"  • Month {month}: {avg_retention[month]:.1f}%")

## 5. Revenue Contribution by Cohort Age

Analyze how much revenue each cohort contributes over time.

In [ ]:
# Query cumulative revenue by cohort
query_revenue = f"""
SELECT
  cohort_month_name,
  months_since_cohort,
  cumulative_revenue,
  cohort_size
FROM `{project_id}.{dataset}.mart_customer_retention`
WHERE cohort_month >= '2017-01-01'
  AND cohort_month <= '2017-03-01'  -- First 3 cohorts for clarity
  AND months_since_cohort <= 12
ORDER BY cohort_month, months_since_cohort
"""

df_revenue = client.query(query_revenue).to_dataframe()

# Calculate revenue per customer
df_revenue["revenue_per_customer"] = df_revenue["cumulative_revenue"] / df_revenue["cohort_size"]

# Plot cumulative revenue curves
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Cumulative revenue
for cohort in df_revenue["cohort_month_name"].unique():
    cohort_data = df_revenue[df_revenue["cohort_month_name"] == cohort]
    ax1.plot(
        cohort_data["months_since_cohort"],
        cohort_data["cumulative_revenue"],
        marker="o",
        linewidth=2,
        label=cohort,
    )

ax1.set_xlabel("Months Since First Order", fontsize=12)
ax1.set_ylabel("Cumulative Revenue (R$)", fontsize=12)
ax1.set_title("Cumulative Revenue by Cohort", fontsize=14, fontweight="bold")
ax1.legend()
ax1.grid(True, alpha=0.3)

# Revenue per customer
for cohort in df_revenue["cohort_month_name"].unique():
    cohort_data = df_revenue[df_revenue["cohort_month_name"] == cohort]
    ax2.plot(
        cohort_data["months_since_cohort"],
        cohort_data["revenue_per_customer"],
        marker="o",
        linewidth=2,
        label=cohort,
    )

ax2.set_xlabel("Months Since First Order", fontsize=12)
ax2.set_ylabel("Revenue per Customer (R$)", fontsize=12)
ax2.set_title("Revenue per Customer by Cohort", fontsize=14, fontweight="bold")
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 6. Actionable Recommendations

Generate data-driven recommendations based on cohort analysis.

In [ ]:
print("\n" + "=" * 80)
print("🎯 ACTIONABLE RECOMMENDATIONS")
print("=" * 80)

# Find best and worst performing cohorts
best_ltv_cohort = cohort_summary.loc[cohort_summary["customer_ltv"].idxmax()]
worst_ltv_cohort = cohort_summary.loc[cohort_summary["customer_ltv"].idxmin()]

print("\n1️⃣ ACQUISITION STRATEGY:")
print(
    f"   • Best cohort: {best_ltv_cohort['cohort_month_name']} (LTV: R${best_ltv_cohort['customer_ltv']:.2f})"
)
print(f"   • Recommendation: Analyze marketing campaigns from this period")
print(f"   • Action: Replicate successful acquisition channels and messaging")

avg_loyal = cohort_summary["loyal_pct"].mean()
print("\n2️⃣ RETENTION IMPROVEMENT:")
print(f"   • Current loyal customer rate: {avg_loyal:.1f}%")
if avg_loyal < 30:
    print(f"   • Target: Increase to 30% through loyalty programs")
    print(f"   • Action: Implement points/rewards system for repeat purchases")
else:
    print(f"   • Action: Maintain current retention programs")

# Calculate churn point
if "avg_retention" in locals():
    print("\n3️⃣ CHURN PREVENTION:")
    if 3 in avg_retention.index and avg_retention[3] < 50:
        print(f"   • Critical drop at month 3: {avg_retention[3]:.1f}% retention")
        print(f"   • Action: Launch win-back campaign at 60-90 days post-purchase")
        print(f"   • Tactic: Personalized email with discount code")

print("\n4️⃣ SEGMENT FOCUS:")
one_time_pct = cohort_summary["one_time_pct"].mean()
if one_time_pct > 50:
    print(f"   • {one_time_pct:.1f}% are one-time customers")
    print(f"   • Action: Create 'second purchase' incentive campaign")
    print(f"   • Target: Reduce one-time customers to <40%")

print("\n5️⃣ LTV OPTIMIZATION:")
avg_ltv = cohort_summary["customer_ltv"].mean()
print(f"   • Current average LTV: R${avg_ltv:.2f}")
print(f"   • Potential with 20% improvement: R${avg_ltv * 1.2:.2f}")
print(f"   • Action: Upsell/cross-sell campaigns for repeat customers")
print(f"   • Focus: Product bundling and category expansion")

print("\n" + "=" * 80)
print("✅ Analysis Complete - Ready for Stakeholder Presentation")
print("=" * 80)